In [1]:
# --- 0. 安装必要的库 ---
# 在Kaggle环境中，您需要先安装sentence-transformers
# 确保您的Kaggle Notebook已在 "Settings" -> "Internet" 中开启网络
!pip install -q sentence-transformers transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 14.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's depen

In [2]:
# --- 1. 导入库 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.colors as mcolors
import os
import sys
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE  # 用于降维可视化
from sklearn.feature_extraction.text import TfidfVectorizer  # 用于提取关键词
try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    print("错误：无法导入 SentenceTransformer。请先在 Kaggle 中执行：!pip install -q sentence-transformers transformers")
    sys.exit()

# --- 2. 字体与文件路径 ---
font_path_chinese = "/kaggle/input/nsfc2025-management/simkai.ttf"
font_path_english = "/kaggle/input/nsfc2025-management/times.ttf"
data_file = "/kaggle/input/nsfc2025-management/nsfc-2025-management-title-fenci.csv"

# 字体检查与注册
if os.path.exists(font_path_chinese):
    ch_font = fm.FontProperties(fname=font_path_chinese)
    fm.fontManager.addfont(font_path_chinese)
else:
    print(f"警告: 未找到中文字体 '{font_path_chinese}'，中文显示可能不完整。")
    ch_font = fm.FontProperties()

if os.path.exists(font_path_english):
    en_font = fm.FontProperties(fname=font_path_english)
    fm.fontManager.addfont(font_path_english)
else:
    print(f"警告: 未找到英文字体 '{font_path_english}'，英文字体将使用默认。")
    en_font = fm.FontProperties()

plt.rcParams['axes.unicode_minus'] = False

# --- 3. 加载与校验数据（仅两列：项目名称、项目名称分词） ---
print(f"正在从 '{data_file}' 加载数据...")
try:
    df = pd.read_csv(data_file, dtype=str, encoding="utf-8")
except FileNotFoundError:
    print(f"错误: 数据文件 '{data_file}' 未找到。")
    sys.exit()
except Exception as e:
    print(f"读取 CSV 文件时出错: {e}")
    sys.exit()

# 去除列名两端空格，兼容“项目名称分词 ”等情况
df.columns = [str(c).strip() for c in df.columns]

required_cols = {"项目名称", "项目名称分词"}
if not required_cols.issubset(set(df.columns)):
    print(f"错误：数据中缺少必需列。需要列：{required_cols}，当前列：{list(df.columns)}")
    sys.exit()

# 仅保留两列并去除缺失
df = df.loc[:, ["项目名称", "项目名称分词"]].dropna(subset=["项目名称", "项目名称分词"]).copy()

# 准备编码文本（可切换为“项目名称分词”）
titles = df["项目名称"].astype(str).tolist()
print(f"已加载 {len(titles)} 条项目标题。")

# --- 4. 加载模型并生成编码 ---
model_name = "jinaai/jina-embeddings-v2-base-zh"
print(f"正在从 Hugging Face 加载模型：{model_name} ...")
try:
    model = SentenceTransformer(model_name, trust_remote_code=True)
    print("模型加载成功。")
except Exception as e:
    print(f"加载模型时出错: {e}")
    sys.exit()

print(f"正在为 {len(titles)} 个标题生成编码（可能需要一些时间）...")
embeddings = model.encode(titles, show_progress_bar=True)
print(f"编码完成。shape = {embeddings.shape}")

# --- 5. K-means 聚类（K=7） ---
K = 7
print(f"正在执行 K-means 聚类, K={K} ...")
kmeans = KMeans(n_clusters=K, random_state=2025, n_init=10)
labels = kmeans.fit_predict(embeddings)

# 将聚类标签添加回 DataFrame
df["cluster"] = labels
print(f"聚类完成。各簇大小分布：\n{df['cluster'].value_counts().sort_index()}")

# --- 6. t-SNE 降维可视化 ---
print("正在执行 t-SNE 降维（这可能需要几分钟）...")
tsne = TSNE(n_components=2, random_state=2025, perplexity=30, n_iter=1000)
tsne_results = tsne.fit_transform(embeddings)
df["tsne_x"] = tsne_results[:, 0]
df["tsne_y"] = tsne_results[:, 1]
print("t-SNE 降维完成。")

# --- 7. 绘制聚类结果图（t-SNE 散点图） ---
print("正在绘制 t-SNE 聚类散点图...")
fig, ax = plt.subplots(figsize=(16, 12))

palette = list(mcolors.TABLEAU_COLORS.values())
for i in range(K):
    sub = df[df["cluster"] == i]
    ax.scatter(
        sub["tsne_x"],
        sub["tsne_y"],
        color=palette[i % len(palette)],
        label=f"簇 {i}",
        alpha=0.75,
        s=50
    )

ax.set_title(f"项目 K-means 聚类结果 (K={K}, t-SNE 降维)", fontproperties=ch_font, fontsize=20, pad=20)
ax.set_xlabel("t-SNE Component 1", fontproperties=ch_font, fontsize=14)
ax.set_ylabel("t-SNE Component 2", fontproperties=ch_font, fontsize=14)
leg = ax.legend(prop=ch_font, fontsize=12, title="聚类簇")
plt.setp(leg.get_title(), fontproperties=ch_font)
plt.setp(ax.get_xticklabels(), fontproperties=en_font, fontsize=12)
plt.setp(ax.get_yticklabels(), fontproperties=en_font, fontsize=12)
ax.grid(True, linestyle="--", alpha=0.5)

fig.tight_layout()
out_fig = "kmeans_tsne_k7.png"
try:
    fig.savefig(out_fig, dpi=300, bbox_inches="tight")
    print(f"图表已保存：{out_fig}（/kaggle/working/）")
except Exception as e:
    print(f"保存图表时出错：{e}")
plt.show()
plt.close(fig)

# --- 8. 为每个簇提取 10 个代表性关键词（基于“项目名称分词”） ---
print("\n" + "="*50)
print("正在为每个簇提取代表性关键词（TF-IDF）...")
print("="*50)

# 可按需调整停用词
my_stop_words = ['研究', '机制', '基于', '影响', '优化', '效应', '视角', '策略', 
                 '驱动', '理论', '机理', '路径', '动态', '分析', '应用', '作用', 
                 '模型', '方法', '评估', '及其', '设计', '中国', '模式', '形成', 
                 '发展', '结构', '系统', '背景', '提升', '后果', '实证', '考虑', 
                 '问题', '建模', '面向', '测度', '效果', '预测', '识别', '构建', 
                 '关键', '传导', '——']

# 将每个簇内的“项目名称分词”合并为长文本
cluster_texts = df.groupby("cluster")["项目名称分词"].apply(lambda x: " ".join(map(str, x))).reindex(range(K)).tolist()

tfidf_vec = TfidfVectorizer(stop_words=my_stop_words, max_features=1000, ngram_range=(1, 2))
tfidf_matrix = tfidf_vec.fit_transform(cluster_texts)
feature_names = tfidf_vec.get_feature_names_out()

for i in range(K):
    scores = tfidf_matrix[i].toarray().ravel()
    top_idx = scores.argsort()[-10:][::-1]
    top_words = [feature_names[j] for j in top_idx]
    print(f"\n--- 簇 {i}（{(df['cluster']==i).sum()} 项）Top 10 关键词 ---")
    print(" | ".join(top_words))

print("\n" + "="*50)
print("所有任务完成。")


2025-10-23 07:30:25.363825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761204625.585116      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761204625.647578      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


正在从 '/kaggle/input/nsfc2025-management/nsfc-2025-management-title-fenci.csv' 加载数据...
已加载 927 条项目标题。
正在从 Hugging Face 加载模型：jinaai/jina-embeddings-v2-base-zh ...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

模型加载成功。
正在为 927 个标题生成编码（可能需要一些时间）...


Batches:   0%|          | 0/29 [00:00<?, ?it/s]

编码完成。shape = (927, 768)
正在执行 K-means 聚类, K=7 ...
聚类完成。各簇大小分布：
cluster
0    135
1    127
2    157
3    100
4    173
5    105
6    130
Name: count, dtype: int64
正在执行 t-SNE 降维（这可能需要几分钟）...
t-SNE 降维完成。
正在绘制 t-SNE 聚类散点图...
图表已保存：kmeans_tsne_k7.png（/kaggle/working/）



正在为每个簇提取代表性关键词（TF-IDF）...

--- 簇 0（135 项）Top 10 关键词 ---
气候 | 绿色 | 低碳 | 生态 | 转型 | 政策 | 能源 | 协同 | 风险 | 企业

--- 簇 1（127 项）Top 10 关键词 ---
政策 | 健康 | 行为 | 社会 | 农村 | 教育 | 干预 | 居民 | 人口 | 福利

--- 簇 2（157 项）Top 10 关键词 ---
创新 | 企业 | 技术 | 数字 | 供应链 | 产业 | 经济 | 政策 | 核心 | 数字 技术

--- 簇 3（100 项）Top 10 关键词 ---
调度 | 服务 | 城市 | 供应链 | 库存 | 配送 | 交通 | 协同 | 智能 | 充电

--- 簇 4（173 项）Top 10 关键词 ---
智能 | 行为 | 人机 | 团队 | 人工 | 人工 智能 | 决策 | 干预 | ai | 协作

--- 簇 5（105 项）Top 10 关键词 ---
银行 | 债务 | 风险 | 信息 | 政策 | 货币 | 经济 | 行为 | 企业 | 市场

--- 簇 6（130 项）Top 10 关键词 ---
数据 | 统计 | 风险 | 模态 | 推断 | 算法 | 高维 | 定价 | 学习 | 统计 推断

所有任务完成。
